In [1]:
import json
from ecnet.datasets import load_cp

def get_property(compounds: list, property: str, lim: int = None) -> tuple:

    prop_vals = []
    smiles = []
    for c in compounds:
        try:
            _val = [float(c['properties'][property]['value'])]
        except KeyError:
            continue
        if lim is not None:
            if _val[0] > lim:
                continue
        prop_vals.append(_val)
        smiles.append(c['canonical_smiles'])
    return (prop_vals, smiles)


with open('compounds.json', 'r') as jfile:
    compounds = json.load(jfile)
jfile.close()

cn, smiles_cn = get_property(compounds, 'cetane_number', 100)
ysi, smiles_ysi = get_property(compounds, 'ysi_unified')
lhv, smiles_lhv = get_property(compounds, 'lower_heating_value')
kv, smiles_kv = get_property(compounds, 'kinematic_viscosity')
smiles_cp, cp = load_cp()
fp, smiles_fp = get_property(compounds, 'flash_point')

print(len(cn), len(ysi), len(lhv), len(kv), len(cp), len(fp))

391 545 385 205 43 255


In [2]:
from sklearn.model_selection import train_test_split

cn_train, cn_test, smiles_cn_train, smiles_cn_test = train_test_split(cn, smiles_cn, test_size=0.2, random_state=42)
ysi_train, ysi_test, smiles_ysi_train, smiles_ysi_test = train_test_split(ysi, smiles_ysi, test_size=0.2, random_state=42)
lhv_train, lhv_test, smiles_lhv_train, smiles_lhv_test = train_test_split(lhv, smiles_lhv, test_size=0.2, random_state=42)
kv_train, kv_test, smiles_kv_train, smiles_kv_test = train_test_split(kv, smiles_kv, test_size=0.2, random_state=42)
cp_train, cp_test, smiles_cp_train, smiles_cp_test = train_test_split(cp, smiles_cp, test_size=0.2, random_state=42)
fp_train, fp_test, smiles_fp_train, smiles_fp_test = train_test_split(fp, smiles_fp, test_size=0.2, random_state=42)

In [3]:
from ecnet.datasets import QSPRDataset

ds_cn_train = QSPRDataset(smiles_cn_train, cn_train, backend='alvadesc')
ds_cn_test = QSPRDataset(smiles_cn_test, cn_test, backend='alvadesc')

ds_ysi_train = QSPRDataset(smiles_ysi_train, ysi_train, backend='alvadesc')
ds_ysi_test = QSPRDataset(smiles_ysi_test, ysi_test, backend='alvadesc')

ds_lhv_train = QSPRDataset(smiles_lhv_train, lhv_train, backend='alvadesc')
ds_lhv_test = QSPRDataset(smiles_lhv_test, lhv_test, backend='alvadesc')

ds_kv_train = QSPRDataset(smiles_kv_train, kv_train, backend='alvadesc')
ds_kv_test = QSPRDataset(smiles_kv_test, kv_test, backend='alvadesc')

ds_cp_train = QSPRDataset(smiles_cp_train, cp_train, backend='alvadesc')
ds_cp_test = QSPRDataset(smiles_cp_test, cp_test, backend='alvadesc')

ds_fp_train = QSPRDataset(smiles_fp_train, fp_train, backend='alvadesc')
ds_fp_test = QSPRDataset(smiles_fp_test, fp_test, backend='alvadesc')

In [4]:
from ecnet.tasks import select_rfr

idx_cn, _ = select_rfr(ds_cn_train, 0.99, n_estimators=100)
idx_ysi, _ = select_rfr(ds_ysi_train, 0.99, n_estimators=100)
idx_kv, _ = select_rfr(ds_kv_train, 0.99, n_estimators=100)
idx_cp, _ = select_rfr(ds_cp_train, 0.99, n_estimators=100)
idx_lhv, _ = select_rfr(ds_lhv_train, 0.99, n_estimators=100)
idx_fp, _ = select_rfr(ds_fp_train, 0.99, n_estimators=100)

ds_cn_train.set_desc_index(idx_cn)
ds_cn_test.set_desc_index(idx_cn)

ds_ysi_train.set_desc_index(idx_ysi)
ds_ysi_test.set_desc_index(idx_ysi)

ds_kv_train.set_desc_index(idx_kv)
ds_kv_test.set_desc_index(idx_kv)

ds_cp_train.set_desc_index(idx_cp)
ds_cp_test.set_desc_index(idx_cp)

ds_lhv_train.set_desc_index(idx_lhv)
ds_lhv_test.set_desc_index(idx_lhv)

ds_fp_train.set_desc_index(idx_fp)
ds_fp_test.set_desc_index(idx_fp)

print(ds_cn_train.desc_vals.shape)
print(ds_ysi_train.desc_vals.shape)
print(ds_kv_train.desc_vals.shape)
print(ds_cp_train.desc_vals.shape)
print(ds_lhv_train.desc_vals.shape)
print(ds_fp_train.desc_vals.shape)

with open('models/cn/desc.txt', 'w', encoding='utf8') as f:
    f.writelines([str(i) + '\n' for i in idx_cn])
f.close()
with open('models/ysi/desc.txt', 'w', encoding='utf8') as f:
    f.writelines([str(i) + '\n' for i in idx_ysi])
f.close()
with open('models/kv/desc.txt', 'w', encoding='utf8') as f:
    f.writelines([str(i) + '\n' for i in idx_kv])
f.close()
with open('models/cp/desc.txt', 'w', encoding='utf8') as f:
    f.writelines([str(i) + '\n' for i in idx_cp])
f.close()
with open('models/lhv/desc.txt', 'w', encoding='utf8') as f:
    f.writelines([str(i) + '\n' for i in idx_lhv])
f.close()
with open('models/fp/desc.txt', 'w', encoding='utf8') as f:
    f.writelines([str(i) + '\n' for i in idx_fp])
f.close()

torch.Size([312, 661])
torch.Size([436, 303])
torch.Size([164, 568])
torch.Size([34, 195])
torch.Size([308, 101])
torch.Size([204, 745])


In [5]:
from ecnet import ECNet
from sklearn.metrics import median_absolute_error, r2_score

mae_cn = []
mae_ysi = []
mae_kv = []
mae_cp = []
mae_lhv = []
mae_fp = []

r2_cn = []
r2_ysi = []
r2_kv = []
r2_cp = []
r2_lhv = []
r2_fp = []


for i in range(25):

    _model_cn = ECNet(ds_cn_train.desc_vals.shape[1], ds_cn_train.target_vals.shape[1], 512, 2)
    _model_ysi = ECNet(ds_ysi_train.desc_vals.shape[1], ds_ysi_train.target_vals.shape[1], 512, 2)
    _model_kv = ECNet(ds_kv_train.desc_vals.shape[1], ds_kv_train.target_vals.shape[1], 512, 2)
    _model_cp = ECNet(ds_cp_train.desc_vals.shape[1], ds_cp_train.target_vals.shape[1], 512, 2)
    _model_lhv = ECNet(ds_lhv_train.desc_vals.shape[1], ds_lhv_train.target_vals.shape[1], 512, 2)
    _model_fp = ECNet(ds_fp_train.desc_vals.shape[1], ds_fp_train.target_vals.shape[1], 512, 2)

    print(f'CN {i}...')
    _model_cn.fit(
        dataset=ds_cn_train,
        epochs=512,
        valid_size=0.33,
        shuffle=True,
        random_state=None,
        batch_size=32,
        patience=32,
        lr=0.001
    )
    print(f'YSI {i}...')
    _model_ysi.fit(
        dataset=ds_ysi_train,
        epochs=512,
        valid_size=0.33,
        shuffle=True,
        random_state=None,
        batch_size=32,
        patience=32,
        lr=0.001
    )
    print(f'KV {i}...')
    _model_kv.fit(
        dataset=ds_kv_train,
        epochs=512,
        valid_size=0.33,
        shuffle=True,
        random_state=None,
        batch_size=16,
        patience=32,
        lr=0.001
    )
    print(f'CP {i}...')
    _model_cp.fit(
        dataset=ds_cp_train,
        epochs=512,
        valid_size=0.33,
        shuffle=True,
        random_state=None,
        batch_size=4,
        patience=32,
        lr=0.001
    )
    print(f'LHV {i}...')
    _model_lhv.fit(
        dataset=ds_lhv_train,
        epochs=512,
        valid_size=0.33,
        shuffle=True,
        random_state=None,
        batch_size=32,
        patience=32,
        lr=0.001
    )
    print(f'FP {i}...')
    _model_fp.fit(
        dataset=ds_fp_train,
        epochs=512,
        valid_size=0.33,
        shuffle=True,
        random_state=None,
        batch_size=32,
        patience=32,
        lr=0.001
    )

    pred_cn = _model_cn(ds_cn_test.desc_vals).detach().numpy()
    pred_ysi = _model_ysi(ds_ysi_test.desc_vals).detach().numpy()
    pred_kv = _model_kv(ds_kv_test.desc_vals).detach().numpy()
    pred_cp = _model_cp(ds_cp_test.desc_vals).detach().numpy()
    pred_lhv = _model_lhv(ds_lhv_test.desc_vals).detach().numpy()
    pred_fp = _model_fp(ds_fp_test.desc_vals).detach().numpy()

    mae_cn.append(median_absolute_error(cn_test, pred_cn))
    mae_ysi.append(median_absolute_error(ysi_test, pred_ysi))
    mae_kv.append(median_absolute_error(kv_test, pred_kv))
    mae_cp.append(median_absolute_error(cp_test, pred_cp))
    mae_lhv.append(median_absolute_error(lhv_test, pred_lhv))
    mae_fp.append(median_absolute_error(fp_test, pred_fp))

    r2_cn.append(r2_score(cn_test, pred_cn))
    r2_ysi.append(r2_score(ysi_test, pred_ysi))
    r2_kv.append(r2_score(kv_test, pred_kv))
    r2_cp.append(r2_score(cp_test, pred_cp))
    r2_lhv.append(r2_score(lhv_test, pred_lhv))
    r2_fp.append(r2_score(fp_test, pred_fp))

    _model_cn.save(f'models/cn/cn_{i}.pt')
    _model_ysi.save(f'models/ysi/ysi_{i}.pt')
    _model_kv.save(f'models/kv/kv_{i}.pt')
    _model_cp.save(f'models/cp/cp_{i}.pt')
    _model_lhv.save(f'models/lhv/lhv_{i}.pt')
    _model_fp.save(f'models/fp/fp_{i}.pt')


CN 0...
YSI 0...
KV 0...
CP 0...
LHV 0...
FP 0...
CN 1...
YSI 1...
KV 1...
CP 1...
LHV 1...
FP 1...
CN 2...
YSI 2...
KV 2...
CP 2...
LHV 2...
FP 2...
CN 3...
YSI 3...
KV 3...
CP 3...
LHV 3...
FP 3...
CN 4...
YSI 4...
KV 4...
CP 4...
LHV 4...
FP 4...
CN 5...
YSI 5...
KV 5...
CP 5...
LHV 5...
FP 5...
CN 6...
YSI 6...
KV 6...
CP 6...
LHV 6...
FP 6...
CN 7...
YSI 7...
KV 7...
CP 7...
LHV 7...
FP 7...
CN 8...
YSI 8...
KV 8...
CP 8...
LHV 8...
FP 8...
CN 9...
YSI 9...
KV 9...
CP 9...
LHV 9...
FP 9...
CN 10...
YSI 10...
KV 10...
CP 10...
LHV 10...
FP 10...
CN 11...
YSI 11...
KV 11...
CP 11...
LHV 11...
FP 11...
CN 12...
YSI 12...
KV 12...
CP 12...
LHV 12...
FP 12...
CN 13...
YSI 13...
KV 13...
CP 13...
LHV 13...
FP 13...
CN 14...
YSI 14...
KV 14...
CP 14...
LHV 14...
FP 14...
CN 15...
YSI 15...
KV 15...
CP 15...
LHV 15...
FP 15...
CN 16...
YSI 16...
KV 16...
CP 16...
LHV 16...
FP 16...
CN 17...
YSI 17...
KV 17...
CP 17...
LHV 17...
FP 17...
CN 18...
YSI 18...
KV 18...
CP 18...
LHV 18...
FP 18

In [6]:
import numpy as np

print(f'CN MAE: {np.mean(mae_cn):.3f} +/- {np.std(mae_cn):.3f}')
print(f'CN R2: {np.mean(r2_cn):.3f} +/- {np.std(r2_cn):.3f}')

print(f'YSI MAE: {np.mean(mae_ysi):.3f} +/- {np.std(mae_ysi):.3f}')
print(f'YSI R2: {np.mean(r2_ysi):.3f} +/- {np.std(r2_ysi):.3f}')

print(f'KV MAE: {np.mean(mae_kv):.3f} +/- {np.std(mae_kv):.3f}')
print(f'KV R2: {np.mean(r2_kv):.3f} +/- {np.std(r2_kv):.3f}')

print(f'CP MAE: {np.mean(mae_cp):.3f} +/- {np.std(mae_cp):.3f}')
print(f'CP R2: {np.mean(r2_cp):.3f} +/- {np.std(r2_cp):.3f}')

print(f'LHV MAE: {np.mean(mae_lhv):.3f} +/- {np.std(mae_lhv):.3f}')
print(f'LHV R2: {np.mean(r2_lhv):.3f} +/- {np.std(r2_lhv):.3f}')

print(f'FP MAE: {np.mean(mae_fp):.3f} +/- {np.std(mae_fp):.3f}')
print(f'FP R2: {np.mean(r2_fp):.3f} +/- {np.std(r2_fp):.3f}')

CN MAE: 4.994 +/- 0.911
CN R2: 0.881 +/- 0.026
YSI MAE: 6.020 +/- 1.555
YSI R2: 0.959 +/- 0.061
KV MAE: 0.073 +/- 0.032
KV R2: 0.926 +/- 0.041
CP MAE: 5.822 +/- 3.465
CP R2: 0.842 +/- 0.093
LHV MAE: 0.704 +/- 0.208
LHV R2: 0.982 +/- 0.005
FP MAE: 8.383 +/- 2.095
FP R2: 0.899 +/- 0.029
